In [ ]:
import pandas as pd
import joblib
import os
import gc

In [ ]:
def predict(data, models):

    model_list = []
    for path in ['models_DART_top56corr/']:
        for fname in os.listdir(path):
            for model_name in models:
                if model_name in fname:
                    model_list.append(path + fname)

    pred_list = []
    for model_path in model_list:
        print(model_path)
        print('{:.02f} MB'.format(os.path.getsize(model_path)/1000000))
        model = joblib.load(model_path)
        pred_list.append(model.predict(data))
    
    return pred_list

In [ ]:
test = pd.read_parquet('Data/test_all_slopes_corr_pcaslope.parquet')
corr_col = test.columns[test.columns.str.startswith('corr_')].to_list()

top_corr =['corr_D_39-B_26','corr_D_48-B_4','corr_S_5-S_24','corr_D_43-D_144','corr_P_2-D_44','corr_R_1-B_4',
 'corr_D_44-B_3','corr_S_5-D_43','corr_S_3-D_62','corr_S_27-B_3','corr_D_46-B_4','corr_D_48-B_9',
 'corr_R_27-D_69','corr_D_47-B_4','corr_D_43-B_4','corr_D_48-D_47','corr_D_39-B_2','corr_D_48-D_39',
 'corr_D_71-D_62','corr_R_27-R_1','corr_D_48-B_3','corr_D_47-D_39','corr_R_6-D_39','corr_D_44-B_4',
 'corr_S_23-D_43','corr_R_27-B_26','corr_S_7-S_3','corr_D_39-B_10','corr_S_3-D_39','corr_D_39-B_3',
 'corr_S_7-S_27','corr_S_27-B_2','corr_P_2-B_4','corr_P_3-D_46','corr_P_2-D_48','corr_S_12-B_4',
 'corr_S_25-B_4','corr_S_19-D_39','corr_R_1-D_39','corr_R_6-D_69','corr_D_39-B_1','corr_D_39-B_15',
 'corr_R_27-D_39','corr_S_23-D_39','corr_R_27-B_5','corr_S_16-D_39','corr_D_39-B_11','corr_P_3-D_47']

corr_to_remove = set(corr_col).difference(set(top_corr))
test.drop(corr_to_remove, axis=1, inplace=True)

models = [
    'fold_0',
    'fold_1',
    'fold_2',
    'fold_3',
    'fold_4',
]

pred_list = predict(test, models)

In [ ]:
column_name = [
    'fold_3',
    'fold_0',
]

pred_df = pd.DataFrame(pred_list).T
pred_df.columns = column_name
pred_df.index = test.index

del test; gc.collect()

In [ ]:
pred_df.to_csv('Output/p_LgbmDART_AggPcaDiffRoundSlopeCorr5for124_56for03_PcaSlope_HT1_folds.csv')
pred_df.mean(axis=1).to_csv('Output/p_LgbmDART_AggPcaDiffRoundSlopeCorr5for124_56for03_PcaSlope_HT1.csv', header=['prediction'])

In [ ]:
p1 = pd.read_csv('Output/p_LgbmDART_AggPcaDiffRoundSlopeCorr5for124_56for03_PcaSlope_HT1.csv', index_col='customer_ID')
p2 = pd.read_csv('Output/lag_submission.csv', index_col='customer_ID').loc[p1.index]

In [ ]:
p_ensemble = 0.35 * p1['prediction'] + 0.65 * p2['prediction']

In [ ]:
p_ensemble.to_csv('Output/p_ensemble_LgbmDART_AggPcaDiffRoundSlopeCorr5for124_56for03_PcaSlope_HT1_0.65lag.csv', header=['prediction'])

In [ ]:
p_ensemble